Problem Statement: <br/>
    
Design and implement a data structure for Least Frequently Used (LFU) cache. <br/>
Implement the LFUCache class: <br/>

    LFUCache(int capacity) Initializes the object with the capacity of the data structure.
    int get(int key) Gets the value of the key if the key exists in the cache. Otherwise, returns -1.
    void put(int key, int value) Sets or inserts the value if the key is not already present. When the cache reaches its capacity, it should invalidate the least frequently used item before inserting a new item. For this problem, when there is a tie (i.e., two or more keys with the same frequency), the least recently used key would be evicted.

Notice that the number of times an item is used is the number of calls to the get and put functions for that item since it was inserted. This number is set to zero when the item is removed. <br/>

Example 1: <br/>
Input: <br/>
["LFUCache", "put", "put", "get", "put", "get", "get", "put", "get", "get", "get"] <br/>
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [3], [4, 4], [1], [3], [4]] <br/>
Output: <br/>
[null, null, null, 1, null, -1, 3, null, -1, 3, 4] <br/>

Explanation: <br/>
LFUCache lfu = new LFUCache(2); <br/>
lfu.put(1, 1); <br/>
lfu.put(2, 2); <br/>
lfu.get(1);      // return 1 <br/>
lfu.put(3, 3);   // evicts key 2 <br/>
lfu.get(2);      // return -1 (not found) <br/>
lfu.get(3);      // return 3 <br/>
lfu.put(4, 4);   // evicts key 1. <br/>
lfu.get(1);      // return -1 (not found) <br/>
lfu.get(3);      // return 3 <br/>
lfu.get(4);      // return 4

# Doubly Linked List, HashMap - O(1) Get, O(N) Put runtime, O(N) space

In [1]:
class DLinkedNode:
    def __init__(self, key:int = 0, value:int = 0):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None
        
class LFUCache:
    
    def _add_node(self, node: DLinkedNode) -> None:
        node.prev = self.head
        node.next = self.head.next
        
        self.head.next.prev = node
        self.head.next = node
        
    def _remove_node(self, node: DLinkedNode):
        prev = node.prev
        after = node.next
        
        prev.next = after
        after.prev = prev
        
    def __init__(self, capacity: int):
        self.cache = {}
        self.capacity = capacity
        self.size = 0
        self.head, self.tail = DLinkedNode(), DLinkedNode()
        self.head.next = self.tail
        self.tail.prev = self.head

    def get(self, key: int) -> int:
        node, freq = self.cache.get(key, [None, 0])
        if self.capacity == 0 or not node:
            return -1
        
        value = node.value
        self._remove_node(node)
        self._add_node(node)
        self.cache[key][1] = freq + 1
        
        return value

    def put(self, key: int, value: int) -> None:
        if self.capacity == 0:
            return
        node, freq = self.cache.get(key, [None, 0])
        if node:
            node.value = value
            self._remove_node(node)
            self._add_node(node)
            self.cache[key][1] = freq + 1
        else: 
            if self.size == self.capacity:
                leastFrequentValue = min(self.cache.values(), key=lambda x: x[1])[1]
                leastFrequentNodes = [value[0] for key, value in self.cache.items() if value[1] == leastFrequentValue]
                
                if len(leastFrequentNodes) == 1:
                    LFR_node = leastFrequentNodes[0]
                else:
                    tail = self.tail
                    while tail.prev:
                        tail = tail.prev
                        if tail in leastFrequentNodes:
                            LFR_node = tail
                            break
                self._remove_node(LFR_node)
                self.cache.pop(LFR_node.key)
                self.size -= 1
                
            node = DLinkedNode(key, value)
            self.cache[key] = [node, 0]
            self._add_node(node)
            self.size += 1

# Circular Linked List, HashMap - O(1) Get, O(1) Put runtime, O(N) space

In [2]:
import collections

class Node:
    def __init__(self, key, val):
        self.key = key
        self.val = val
        self.freq = 1
        self.prev = self.next = None

class DLinkedList:
    """ An implementation of doubly linked list.

    Two APIs provided:
    
    append(node): append the node to the head of the linked list.
    pop(node=None): remove the referenced node. 
                    If None is given, remove the one from tail, which is the least recently used.
                    
    Both operation, apparently, are in O(1) complexity.
    """
    def __init__(self):
        self._sentinel = Node(None, None) # dummy node
        self._sentinel.next = self._sentinel.prev = self._sentinel
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def append(self, node):
        node.next = self._sentinel.next
        node.prev = self._sentinel
        node.next.prev = node
        self._sentinel.next = node
        self._size += 1
    
    def pop(self, node=None):
        if self._size == 0:
            return
        
        if not node:
            node = self._sentinel.prev

        node.prev.next = node.next
        node.next.prev = node.prev
        self._size -= 1
        
        return node
        
class LFUCache:
    def __init__(self, capacity):
        """
        :type capacity: int
        
        Three things to maintain:
        
        1. a dict, named as `self._node`, for the reference of all nodes given key.
           That is, O(1) time to retrieve node given a key.
           
        2. Each frequency has a doubly linked list, store in `self._freq`, where key
           is the frequency, and value is an object of `DLinkedList`
        
        3. The min frequency through all nodes. We can maintain this in O(1) time, taking
           two rules:
           
           Rule 1: Whenever we see the size of the DLinkedList of current min frequency is 0,
                   the min frequency must increment by 1.
           
           Rule 2: Whenever put in a new (key, value), the min frequency must 1 (the new node)
           
        """
        self._size = 0
        self._capacity = capacity
        
        self._node = dict() # key: Node
        self._freq = collections.defaultdict(DLinkedList)
        self._minfreq = 0
        
        
    def _update(self, node):
        """ 
        This is a helper function that used in the following two cases:
        
            1. when `get(key)` is called; and
            2. when `put(key, value)` is called and the key exists.
         
        The common point of these two cases is that:
        
            1. no new node comes in, and
            2. the node is visited one more times -> node.freq changed -> 
               thus the place of this node will change
        
        The logic of this function is:
        
            1. pop the node from the old DLinkedList (with freq `f`)
            2. append the node to new DLinkedList (with freq `f+1`)
            3. if old DlinkedList has size 0 and self._minfreq is `f`,
               update self._minfreq to `f+1`
        
        All of the above opeartions took O(1) time.
        """
        freq = node.freq
        
        self._freq[freq].pop(node)
        if self._minfreq == freq and not self._freq[freq]:
            self._minfreq += 1
        
        node.freq += 1
        freq = node.freq
        self._freq[freq].append(node)
    
    def get(self, key):
        """
        Through checking self._node[key], we can get the node in O(1) time.
        Just performs self._update, then we can return the value of node.
        
        :type key: int
        :rtype: int
        """
        if key not in self._node:
            return -1
        
        node = self._node[key]
        self._update(node)
        return node.val

    def put(self, key, value):
        """
        If `key` already exists in self._node, we do the same operations as `get`, except
        updating the node.val to new value.
        
        Otherwise, the following logic will be performed
        
        1. if the cache reaches its capacity, pop the least frequently used item. (*)
        2. add new node to self._node
        3. add new node to the DLinkedList with frequency 1
        4. reset self._minfreq to 1
        
        (*) How to pop the least frequently used item? Two facts:
        
        1. we maintain the self._minfreq, the minimum possible frequency in cache.
        2. All cache with the same frequency are stored as a DLinkedList, with
           recently used order (Always append at head)
          
        Consequence? ==> The tail of the DLinkedList with self._minfreq is the least
                         recently used one, pop it...
        
        :type key: int
        :type value: int
        :rtype: void
        """
        if self._capacity == 0:
            return
        
        if key in self._node:
            node = self._node[key]
            self._update(node)
            node.val = value
        else:
            if self._size == self._capacity:
                node = self._freq[self._minfreq].pop()
                del self._node[node.key]
                self._size -= 1
                
            node = Node(key, value)
            self._node[key] = node
            self._freq[1].append(node)
            self._minfreq = 1
            self._size += 1

# DefaultDict, OrderedDict - O(1) Get, O(1) Put runtime, O(N) space

In [4]:
from collections import defaultdict
from collections import OrderedDict

class Node:
    def __init__(self, key, val, count):
        self.key = key
        self.val = val
        self.count = count
    
class LFUCache(object):
    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.cap = capacity
        self.key2node = {}
        self.count2node = defaultdict(OrderedDict)
        self.minCount = None
        
    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key not in self.key2node:
            return -1
        
        node = self.key2node[key]
        del self.count2node[node.count][key]
        
        # clean memory
        if not self.count2node[node.count]:
            del self.count2node[node.count]
        
        node.count += 1
        self.count2node[node.count][key] = node
        
        # NOTICE check minCount!!!
        if not self.count2node[self.minCount]:
            self.minCount += 1
            
            
        return node.val
        
    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        if not self.cap:
            return 
        
        if key in self.key2node:
            self.key2node[key].val = value
            self.get(key) # NOTICE, put makes count+1 too
            return
        
        if len(self.key2node) == self.cap:
            # popitem(last=False) is FIFO, like queue
            # it return key and value!!!
            k, n = self.count2node[self.minCount].popitem(last=False) 
            del self.key2node[k] 
        
        self.count2node[1][key] = self.key2node[key] = Node(key, value, 1)
        self.minCount = 1
        return


In [5]:
lfu = LFUCache(2)
lfu.put(1, 1);
lfu.put(2, 2);
print(lfu.get(1)); # return 1
lfu.put(3, 3); # evicts key 2
print(lfu.get(2)); # return -1 (not found)
print(lfu.get(3)); # return 3
lfu.put(4, 4); # evicts key 1.
print(lfu.get(1)); # return -1 (not found)
print(lfu.get(3)); # return 3
print(lfu.get(4)); # return 4

1
-1
3
-1
3
4
